# Exercise 9

## Mashable news stories analysis

### Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
train_df = pd.read_csv(url, index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
train_df.shape

(6000, 61)

In [4]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular']

In [5]:
y.mean()

0.5

In [6]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# grow each tree deep
treereg = DecisionTreeClassifier()
treereg.fit(X_train,y_train)
y_pred=treereg.predict(X_test)
print("Accuracy: ", metrics.accuracy_score(y_test,y_pred))
print("F1 - Score: ", metrics.f1_score(y_test,y_pred))

Accuracy:  0.5453333333333333
F1 - Score:  0.5477453580901857


# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [10]:
import numpy as np
np.random.seed(123)

#print(X_train.shape[0])

n_samples = X_train.shape[0]
n_B = 300

# create 300 bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
print(enumerate(samples))
samples[0:3]

[array([3582, 3454, 1346, ..., 2409, 3823,  992]),
 array([ 554, 1524, 1802, ..., 1314,  549, 1303]),
 array([3259,  595,  126, ...,  246, 2641,  580])]

### prueba unitaria:

In [11]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
treereg = DecisionTreeClassifier(max_depth=None, random_state=123)

# DataFrame for storing predicted price from each tree
y_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])

########ajustando un solo arbol de CLASIFICACION con una sola muestra (prueba unitaria), validando sobre los datos de test (OJO)

X_tr = X_train.iloc[samples[0], :]
y_tr = y_train.iloc[samples[0]]
treereg.fit(X_tr, y_tr)
y_pred= treereg.predict(X_test)
print(y_pred)

[1 1 1 ... 1 0 0]


### Se ajustan los 100 primeros arboles de clasificación:

In [12]:
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]
print(n_samples)
n_B = 100

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
len(samples)

4500


100

In [15]:
from sklearn.tree import DecisionTreeClassifier

# grow each tree deep
treereg = DecisionTreeClassifier(max_depth=None, random_state=123)

# DataFrame for storing predicted price from each tree
y_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])
#print(y_pred)

 #grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    X_tr = X_train.iloc[sample, :]
    y_tr = y_train.iloc[sample]
    treereg.fit(X_tr, y_tr)
    y_pred[i] = treereg.predict(X_test)
    
print(y_pred[1:3])
    

     0   1   2   3   4   5   6   7   8   9   ... 290 291 292 293 294 295 296  \
2185   1   1   1   1   0   1   0   1   1   0 ...   0   1   0   1   1   1   1   
2520   1   0   0   0   1   1   1   1   0   0 ...   1   0   0   0   0   1   1   

     297 298 299  
2185   1   1   1  
2520   0   0   1  

[2 rows x 300 columns]


In [16]:
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]
print(n_samples)
n_B = 300

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
len(samples)

4500


300

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model, metrics
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

# grow each tree deep
treereg = DecisionTreeClassifier(max_depth=None, random_state=123)
treereg2 = DecisionTreeClassifier(max_depth=2, random_state=123)
regr = linear_model.LogisticRegression(solver='lbfgs')


# DataFrame for storing predicted price from each tree
y_pred = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])
#print(y_pred)

#grow one tree for each bootstrap sample and make predictions on testing data
for i, sample in enumerate(samples):
    
    #print(i)
    
    if (i<100): #donde la expresion es verdadera
        #codigo_a_ejecutar
        X_tr = X_train.iloc[sample, :]
        y_tr = y_train.iloc[sample]
        treereg.fit(X_tr, y_tr)
        y_pred[i] = treereg.predict(X_test)
        
        
    if (i>99 & i<200): #donde la expresion es verdadera
            #codigo_a_ejecutar
            X_tr = X_train.iloc[sample, :]
            y_tr = y_train.iloc[sample]
            treereg2.fit(X_tr, y_tr)
            y_pred[i] = treereg.predict(X_test)
        
    if (i>199): #donde la expresion es verdadera
            #codigo_a_ejecutar
            

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            fxn()
            
            X_tr = X_train.iloc[sample, :]
            y_tr = y_train.iloc[sample]
            regr.fit(X_tr, y_tr)
            y_pred[i] = regr.predict(X_test)

#if expresion: #donde la expresion es verdadera
    #codigo_a_ejecutar

In [18]:
print(y_pred[[1,100,200]].iloc[0:3,:]) 
print(len(y_pred))

     1   100 200
1483   1   1   1
2185   1   1   0
2520   0   1   1
1500


# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [23]:
y_pred1 = (y_pred.sum(axis=1) >= (n_B / 2)).astype(np.int)
print(y_pred1.head())

from sklearn import metrics
print(metrics.f1_score(y_pred1, y_test))
metrics.accuracy_score(y_pred1, y_test)

1483    1
2185    1
2520    1
3721    1
3727    0
dtype: int32
0.62936142198815


0.6246666666666667

# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [24]:
valid=pd.DataFrame(pd.concat([y_pred1,y_test],ignore_index=True,axis=1))
valid.rename(columns={0:'pred',1:'true'}, inplace=True)
valid['resta']=valid['pred']-valid['true']
valid.loc[valid['resta']==0].sum()
valid.shape[0]


1500

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score